## Dependencies

In [ ]:
# Install a pip package in the current Jupyter kernel
#!python -m pip install --upgrade pip pandas jupyterthemes biopython lxml
#!jt -t monokai -fs 12 -cellw 80% -T -N -kl
import re
import datetime
import pandas as pd


In [ ]:
def parse_pub_date(pub_date):
    if not pub_date:
        pub_date = ""
    begin = "^"
    end = "$"
    YYYY = "\d{4}"
    MM_DD ="\d{2}"
    MM_ABRV = "\D+"
    YYYY_MM = begin + YYYY + "-{1}" + MM_DD + end
    YYYY_ABRV = begin + YYYY + "-{1}" + MM_ABRV + end
    YYYY_ABRV_DD = begin + YYYY + "-{1}" + MM_ABRV + "-{1}" + MM_DD + end
    months = {"[Jj][Aa][Nn].*": "01", "[Ff][Ee][Bb].*": "02", "[Mm][Aa][Rr].*": "03",
              "[Aa][Pp][Rr].*": "04", "[Mm][Aa][Yy].*": "05", "[Jj][Uu][Nn].*": "06",
              "[Jj][Uu][Ll].*": "07", "[Aa][Uu][Gg].*": "08", "[Ss][Ee][Pp].*": "09",
              "[Oo][Cc][Tt].*": "10", "[Nn][Oo][Vv].*": "11", "[Dd][Ee][Cc].*": "12",
              "[Ww][Ii][Nn].*": "12", "[Ss][Pp][Rr].*": "03", "[Ss][Uu][Mm].*": "06",
              "[Aa][Uu][Tt].*": "09", "[Ff][Aa][Ll].*": "09"}
    MM_DD_YYYY = begin + "\d{1,2}\/{1}\d{1,2}\/{1}\d{4}" + end
    ORDINAL_DATE = begin + "\d{5}" + end
    if re.search(begin + YYYY + end, pub_date): #2023
        pub_date = pub_date + "-01-01"
    elif re.search(YYYY_MM, pub_date): #2023-06
        pub_date = pub_date + "-01"
    elif re.search(YYYY_ABRV, pub_date): #2023-Jun or 2023-Jun-23
        for month_re, month_number in months.items():
            YYYYMM_ABRV = begin + YYYY + "-{1}" + month_re + end
            if re.search(YYYYMM_ABRV, pub_date):
                pub_date = pub_date[:4] + "-" + month_number + "-01"
    elif re.search(YYYY_ABRV_DD, pub_date):
        for month_re, month_number in months.items():
            YYYYMM_ABRVDD = begin + YYYY + "-{1}" + month_re + "-{1}" + MM_DD + end
            if re.search(YYYYMM_ABRVDD, pub_date):
                pub_date = pub_date[:4] + "-" + month_number + "-" + pub_date[-2:]
    elif re.search(MM_DD_YYYY, pub_date): #
        pub_date = pub_date.split("/")
        pub_date = pub_date[2] + "-" + pub_date[0] + "-" + pub_date[1]
    elif re.search(ORDINAL_DATE, pub_date):
        print("Ordinal date: " + pub_date)
        offset = datetime.datetime(1900, 1, 1)
        pub_date = str((offset + datetime.timedelta(days=int(pub_date))).date())
        #pub_date = str(datetime.date.fromordinal(int(pub_date)))
        print("Ordinal date: " + pub_date)
    
    return pub_date

In [ ]:
%%time
xml_file ="pubmed_data.xml"
xslt_file = "PubMed_XML_to_Pandas_Transformer.xsl"
#dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d') #Perhaps for the future
df = pd.read_xml(xml_file, xpath="/Citation", stylesheet=xslt_file, dtype=str)
date_columns = df.filter(like='Date').columns.to_list()
for column in date_columns:
    df[column] = pd.to_datetime(df[column].apply(lambda x: parse_pub_date(x))).dt.strftime('%Y-%m-%d')
df = df.fillna(value="")
#df['PMID'] = df['PMID'].astype(int)
df.to_excel("Transformed_PubMed_Data.xlsx", index=False, header=True)

Wall time: 1.62 s
